In [1]:
from sklearn.utils import shuffle
from pyvi import ViTokenizer #For split vietnamese words
import pandas as pd #For reading xlsx file
from gensim.parsing.preprocessing import strip_non_alphanum, strip_multiple_whitespaces,preprocess_string, split_alphanum, strip_short, strip_numeric
import re
import numpy as np
from scipy.sparse import coo_matrix
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score
import string

In [17]:
# read file and shuffle
doc = []
label = []
with open('data_email.txt','r', encoding= 'utf-8') as f:
    for line in f:
        doc.append(line[:])
doc = shuffle(doc,random_state = 0)
document = []
for d in doc:
    ind = d.find('|')        
    if ind == 5:   
        label.append(1)
    elif ind == 8:
        label.append(0)
    document.append(d[ind+2:])

In [18]:
def raw_text_preprocess(raw):
    raw = raw.lower()
    raw = re.sub(r"(http\S+)|(https\S+)", "URL_Change", raw)
    raw = re.sub(r"(www\S+)|(\S+@\S+)", "URL_Change", raw)
    raw = re.sub(r"([0-9][0-9]/[0-9][0-9]/[0-9][0-9][0-9][0-9])|([0-9][0-9]/[0-9][0-9])|([0-9][0-9]/[0-9][0-9][0-9][0-9])", "DATE_Change", raw)
    raw = re.sub(r"([0-9][0-9]-[0-9][0-9]-[0-9][0-9][0-9][0-9])|([0-9][0-9]-[0-9][0-9])|([0-9][0-9]-[0-9][0-9][0-9][0-9])", "DATE_Change", raw)
    raw = strip_numeric(raw)
    raw = strip_short(raw, minsize=2)
    raw = ViTokenizer.tokenize(raw)
    return raw

In [19]:
#Chia dữ liệu làm 5 đoạn, lấy đoạn thứ i để test và còn lại để train
def divide_fold(i):
    document_test = document[(i-1)*80:i*80]
    label_test = label[(i-1)*80:i*80]
    document_train = document[:(i-1)*80+1]+document[i*80+1:]
    label_train = label[:(i-1)*80+1]+label[i*80+1:]
    return (document_test,label_test,document_train ,label_train)

In [20]:

def dict_word(document):
    res = {}
    words = {}
    conten = 0
    for doc in document:
            token = doc.split(" ")
            for tok in token:
                if tok in res.keys():
                    res[tok] = res[tok] + 1
                else:
                    res.update({tok : 1})
            for key, value in res.items():
                if (value >= 3):
                    words.update({key : value})
            word = {}
            for tok in token:
                if tok in word.keys():
                    word[tok] = word[tok] + 1
                else:
                    word.update({tok : 1})
            conten = conten + len(word)
    words = sorted(words.keys())
    print(len(words))
    return (words, conten)

In [21]:
def read_data(document, label, words, conten):
    word_list = []
#     conten = 0
    for doc in document:
#         token = change(doc)
        token = doc.split(" ")
        word = {}
        for tok in token:
            if tok in word.keys():
                word[tok] = word[tok] + 1
            else:
                word.update({tok : 1})
        word_list.append(word)
    j = 0
    dat = np.zeros((conten, 3), dtype = int)
    for i, word in enumerate(word_list):
        for key, value in word.items():
            if(key in words):
                dat[j, :] = np.array([int(i), int(words.index(key)), int(value)])
                j = j + 1
    data = coo_matrix((dat[:, 2], (dat[:, 0] , dat[:, 1] )),shape=(len(label), len(words)))
    return (data, label)

In [22]:
#Không tiền xử lý
average = 0
for i in range(1,6):
    document_test,label_test,document_train ,label_train = divide_fold(i)
    (words, conten) = dict_word(document_train)
    (train_data, train_label) = read_data(document_train, label_train, words, conten)
    (test_data, test_label) = read_data(document_test, label_test, words, conten)
    clf = MultinomialNB()
    clf.fit(train_data, train_label)
    y_pred = clf.predict(test_data)
    accuracy = accuracy_score(y_pred,test_label)
    print("Fold ",i)
    print('Training size =',train_data.shape[0],'. accuracy =', accuracy*100)
    average = average + accuracy
print("Average accuracy ",average/5*100)

2780
Fold  1
Training size = 327 . accuracy = 82.5
2919
Fold  2
Training size = 327 . accuracy = 82.5
2922
Fold  3
Training size = 327 . accuracy = 83.75
2796
Fold  4
Training size = 327 . accuracy = 83.75
2840
Fold  5
Training size = 327 . accuracy = 85.0
Average accuracy  83.5


In [23]:
# Xử lý dữ liệu trước khi train
document = [raw_text_preprocess(d) for d in document]
average = 0
for i in range(1,6):
    document_test,label_test,document_train ,label_train = divide_fold(i)
    (words, conten) = dict_word(document_train)
    (train_data, train_label) = read_data(document_train, label_train, words, conten)
    (test_data, test_label) = read_data(document_test, label_test, words, conten)
    clf = MultinomialNB()
    clf.fit(train_data, train_label)
    y_pred = clf.predict(test_data)
    accuracy = accuracy_score(y_pred,test_label)
    print("fole ",i,'Training size =',train_data.shape[0],'. accuracy =', accuracy*100)
    average = average + accuracy
print("Average accuracy ",average/5*100)

2137
fole  1 Training size = 327 . accuracy = 83.75
2230
fole  2 Training size = 327 . accuracy = 80.0
2200
fole  3 Training size = 327 . accuracy = 86.25
2143
fole  4 Training size = 327 . accuracy = 85.0
2124
fole  5 Training size = 327 . accuracy = 83.75
Average accuracy  83.75
